# Airline Flight Insights - Neo4j Database Connection

This notebook connects to the Neo4j database and provides utilities to run predefined queries.

In [1]:
from neo4j import GraphDatabase
from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import json
import re

ModuleNotFoundError: No module named 'neo4j'

In [ ]:
# Load environment variables from .env file
load_dotenv(find_dotenv())

# Get Neo4j connection details from environment
NEO4J_URI = os.getenv('NEO4J_URI') or os.getenv('URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME') or os.getenv('USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD') or os.getenv('PASSWORD')

# Get Google API key for Gemini
google_api_key = os.getenv('GOOGLE_API_KEY')

print(f"Loaded config for URI: {NEO4J_URI}")
print(f"Google API key loaded: {'Yes' if google_api_key else 'No'}")

In [ ]:
# Create Neo4j driver
driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# Verify connection
driver.verify_connectivity()
print("Successfully connected to Neo4j database!")

In [ ]:
# Setup Gemini LLM model (using gemini-2.5-flash)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=google_api_key,
    temperature=0  # Lower temperature for more deterministic JSON output
)
print("Gemini LLM model loaded successfully!")

In [ ]:
queries = [
    # Intent 1: Operational Delay Diagnostics
    "MATCH (j:Journey)-[:ON]->(f:Flight)-[:ARRIVES_AT]->(a:Airport) RETURN a.station_code AS destination, SUM(j.arrival_delay_minutes) AS total_delay ORDER BY total_delay DESC LIMIT $x",
    "MATCH (j:Journey)-[:ON]->(f:Flight)-[:ARRIVES_AT]->(a:Airport) RETURN a.station_code AS destination, SUM(j.arrival_delay_minutes) AS total_delay ORDER BY total_delay ASC LIMIT $x",
    "MATCH (j:Journey)-[:ON]->(f:Flight)-[:DEPARTS_FROM]->(a:Airport) RETURN a.station_code AS origin, SUM(j.arrival_delay_minutes) AS total_delay ORDER BY total_delay DESC LIMIT $x",
    "MATCH (j:Journey)-[:ON]->(f:Flight)-[:DEPARTS_FROM]->(a:Airport) RETURN a.station_code AS origin, SUM(j.arrival_delay_minutes) AS total_delay ORDER BY total_delay ASC LIMIT $x",
    "MATCH (o:Airport {station_code: $origin_station_code})<-[:DEPARTS_FROM]-(f:Flight)-[:ARRIVES_AT]->(d:Airport), (j:Journey)-[:ON]->(f) WITH o, d, AVG(j.arrival_delay_minutes) AS avg_delay WHERE avg_delay > $x RETURN o.station_code AS origin, d.station_code AS destination, avg_delay",
    "MATCH (j:Journey {number_of_legs: $x}) RETURN AVG(j.arrival_delay_minutes) AS avg_delay",

    # Intent 2: Service Quality & Product Optimization
    "MATCH (o:Airport)<-[:DEPARTS_FROM]-(f:Flight)-[:ARRIVES_AT]->(d:Airport), (j:Journey {passenger_class: $class_name})-[:ON]->(f) WITH o, d, AVG(j.food_satisfaction_score) AS avg_food_score WHERE avg_food_score < $threshold RETURN o.station_code AS origin, d.station_code AS destination, avg_food_score",
    "MATCH (j:Journey {food_satisfaction_score: 1})-[:ON]->(f:Flight) WHERE j.actual_flown_miles > $x RETURN DISTINCT f.flight_number",

    # Intent 3: Fleet Performance Monitoring
    "MATCH (j:Journey)-[:ON]->(f:Flight) WHERE j.arrival_delay_minutes > $x RETURN f.fleet_type_description AS aircraft_type, COUNT(j) AS delay_frequency ORDER BY delay_frequency DESC LIMIT 1",
    "MATCH (j:Journey)-[:ON]->(f:Flight {fleet_type_description: $x}) RETURN AVG(j.food_satisfaction_score) AS avg_food_score",
    "MATCH (j:Journey)-[:ON]->(f:Flight {fleet_type_description: $x}) RETURN AVG(j.actual_flown_miles) AS avg_miles",
    "MATCH (j:Journey)-[:ON]->(f:Flight {fleet_type_description: $x}) WITH COUNT(j) AS total_flights, COUNT(CASE WHEN j.arrival_delay_minutes < 0 THEN 1 END) AS early_flights RETURN (TOFLOAT(early_flights) / total_flights) * 100 AS early_arrival_percentage",

    # Intent 4: High-Value Customer (Loyalty) Retention
    "MATCH (p:Passenger {loyalty_program_level: $loyalty_program_level})-[:TOOK]->(j:Journey) RETURN AVG(j.arrival_delay_minutes) AS avg_delay",
    "MATCH (p:Passenger {loyalty_program_level: $loyalty_program_level})-[:TOOK]->(j:Journey) WHERE j.arrival_delay_minutes > $x RETURN p.record_locator AS passenger_id, j.arrival_delay_minutes AS delay",

    # Intent 5: Demographic Market Analysis
    "MATCH (p:Passenger {generation: $generation})-[:TOOK]->(j:Journey)-[:ON]->(f:Flight) WHERE j.actual_flown_miles > $threshold RETURN f.fleet_type_description AS aircraft_type, COUNT(f) AS usage_count ORDER BY usage_count DESC LIMIT 1",
    "MATCH (p:Passenger {generation: $generation})-[:TOOK]->(j:Journey)-[:ON]->(f:Flight) RETURN f.fleet_type_description AS fleet_type, COUNT(f) AS usage_count ORDER BY usage_count DESC LIMIT 1",
    "MATCH (p:Passenger {generation: $generation})-[:TOOK]->(j:Journey)-[:ON]->(f:Flight)-[:ARRIVES_AT]->(a:Airport) RETURN a.station_code AS destination, COUNT(p) AS passenger_volume ORDER BY passenger_volume DESC LIMIT $x"
]

query_descriptions = [
    # Intent 1: Operational Delay Diagnostics
    "Identify the top ${x} destination stations with the highest accumulated arrival delay minutes.",
    "Identify the top ${x} destination stations with the lowest accumulated arrival delay minutes.",
    "Identify the top ${x} origin stations with the highest accumulated arrival delay minutes.",
    "Identify the top ${x} origin stations with the lowest accumulated arrival delay minutes.",
    "Find routes from the origin station ${origin_station_code} where the average arrival delay exceeds ${x} minutes.",
    "Calculate the average arrival delay for flights consisting of exactly ${x} legs.",

    # Intent 2: Service Quality & Product Optimization
    "Identify routes for the passenger class ${class_name} where the average food satisfaction score is below ${threshold}.",
    "List the flight numbers for journeys longer than ${x} miles where the food satisfaction score was 1.",

    # Intent 3: Fleet Performance Monitoring
    "Identify the aircraft type that has the highest frequency of arrival delays greater than ${x} minutes.",
    "Calculate the average food satisfaction score for passengers flying on the ${x} fleet.",
    "Calculate the average actual flown miles for the ${x} fleet.",
    "Calculate the percentage of early arrivals for the ${x} fleet.",

    # Intent 4: High-Value Customer (Loyalty) Retention
    "Calculate the average arrival delay experienced by passengers with the loyalty level ${loyalty_program_level}.",
    "Find the record locators for passengers with loyalty level ${loyalty_program_level} who experienced a delay greater than ${x} minutes.",

    # Intent 5: Demographic Market Analysis
    "Identify the most common aircraft type used by the ${generation} generation for journeys exceeding ${threshold} miles.",
    "Identify the most frequently used fleet type for the ${generation} generation.",
    "Identify the top ${x} destination stations for the ${generation} generation based on passenger volume."
]

print(f"Loaded {len(queries)} queries")

In [ ]:
def run_query(query_index: int, **params) -> list:
    """
    Run a query from the queries list by index with the provided parameters.
    
    Args:
        query_index: The index of the query in the queries list (0-based)
        **params: Keyword arguments for query parameters (e.g., x=5, origin_station_code='LAX')
    
    Returns:
        List of records as dictionaries
    """
    if query_index < 0 or query_index >= len(queries):
        raise ValueError(f"Query index {query_index} is out of range. Valid range: 0-{len(queries)-1}")
    
    query = queries[query_index]
    
    with driver.session() as session:
        result = session.run(query, **params)
        records = [record.data() for record in result]
    
    return records

## LLM-Powered Context Retrieval

The following functions use Gemini to intelligently select relevant queries based on user prompts and format results as context.

In [ ]:
def get_context(prompt: str) -> list:
    """
    Use Gemini LLM to identify relevant queries and extract parameters from user prompt.
    
    Args:
        prompt: User's natural language question
        
    Returns:
        List of dicts with 'query_index' and 'params' keys
    """
    # Build query descriptions with readable parameter placeholders
    safe_descriptions = [desc.replace('${', '<').replace('}', '>') for desc in query_descriptions]
    query_list = "\n".join([f"{i}: {desc}" for i, desc in enumerate(safe_descriptions)])
    num_queries = len(queries) - 1
    
    # Build prompt using string concatenation to avoid f-string escaping issues
    full_prompt = (
        "You are an expert at analyzing user questions about airline flight data and matching them to database queries.\n\n"
        "Available queries (index: description):\n"
        + query_list + "\n\n"
        "Your task:\n"
        "1. Analyze the user's question below\n"
        "2. Identify which query indices (0-" + str(num_queries) + ") would provide useful context\n"
        "3. Extract ALL required parameters from the question for each query\n\n"
        "Parameter reference (shown as <param_name> in queries):\n"
        "- x: a number (count, limit, threshold). Default: 5 for counts, 30 for delays\n"
        "- origin_station_code: airport code like 'LAX', 'ORD', 'JFK'\n"
        "- class_name: 'Economy', 'Business', or 'First'\n"
        "- threshold: numeric value. Default: 1000 for miles\n"
        "- loyalty_program_level: 'Gold', 'Silver', 'Platinum'\n"
        "- generation: 'Millennial', 'Gen X', 'Baby Boomer', 'Gen Z'\n\n"
        'CRITICAL: Return ONLY a valid JSON array. No markdown, no explanation.\n'
        'Format: [{"query_index": 0, "params": {"x": 3}}]\n\n'
        "User question: " + prompt + "\n\n"
        "JSON response:"
    )

    # Use Gemini to get the response
    response = llm.invoke(full_prompt)
    response_text = response.content.strip()
    
    # Clean up response - remove markdown code blocks if present
    response_text = response_text.replace('```json', '').replace('```', '').strip()
    
    # Try to find and parse JSON array
    json_match = re.search(r'\[.*\]', response_text, re.DOTALL)
    if json_match:
        try:
            result = json.loads(json_match.group())
            return result
        except json.JSONDecodeError as e:
            print(f"JSON parse error: {e}")
            print(f"Attempted to parse: {json_match.group()[:200]}")
    
    print(f"Warning: Could not parse LLM response: {response_text[:300]}")
    return []

In [ ]:
def format_query_result(query_index: int, **params) -> str:
    """
    Run a query and format the result as context.
    
    Args:
        query_index: Index of the query to run
        **params: Parameters for the query
        
    Returns:
        Formatted string with query description and results
    """
    if query_index < 0 or query_index >= len(queries):
        return f"Error: Query index {query_index} is out of range."
    
    # Get the description and fill in parameters
    description = query_descriptions[query_index]
    for param_name, param_value in params.items():
        description = description.replace(f"${{{param_name}}}", str(param_value))
    
    # Run the query
    try:
        results = run_query(query_index, **params)
    except Exception as e:
        return f"The answer for \"{description}\" could not be retrieved due to an error: {str(e)}"
    
    # Format the results
    if not results:
        return f"The answer for \"{description}\" is: No data found."
    
    formatted_results = []
    for record in results:
        parts = []
        for key, value in record.items():
            if isinstance(value, float):
                parts.append(f"{key}: {value:.2f}")
            else:
                parts.append(f"{key}: {value}")
        formatted_results.append("  - " + ", ".join(parts))
    
    result_text = "\n".join(formatted_results)
    return f"The answer for \"{description}\" is:\n{result_text}"

## Example: Using LLM Context Functions

The following cells demonstrate how to use `get_context` and `format_query_result`.

In [ ]:
# Example: Get context for a user question
user_question = "What are the top 3 airports with the most delays?"

print(f"User question: {user_question}\n")
print("Analyzing question with Gemini LLM...\n")

# Get relevant queries and parameters
context_queries = get_context(user_question)
print(f"Identified queries: {context_queries}\n")

# Format each result as context
print("=" * 60)
print("RETRIEVED CONTEXT:")
print("=" * 60)
for cq in context_queries:
    context = format_query_result(cq["query_index"], **cq["params"])
    print(context)
    print()

In [ ]:
# Example 2: More complex question
user_question = "How do Millennials travel compared to Gen X? Show me top 5 destinations for each."

print(f"User question: {user_question}\n")
print("Analyzing question with Gemini LLM...\n")

context_queries = get_context(user_question)
print(f"Identified queries: {context_queries}\n")

print("=" * 60)
print("RETRIEVED CONTEXT:")
print("=" * 60)
for cq in context_queries:
    context = format_query_result(cq["query_index"], **cq["params"])
    print(context)
    print()

## Legacy Example Queries

Direct query execution examples.

In [ ]:
# Example 1: Query 0 - Top destinations by total delay
results = run_query(0, x=5)
print("Top 5 destinations by total delay:")
for r in results:
    print(f"  {r['destination']}: {r['total_delay']} minutes")

In [ ]:
# Close driver when done (run this cell when finished)
# driver.close()
# print("Driver closed.")